In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
head = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36"}

In [3]:
newsData = list()
for i in range(1,647):
    url = "https://www.politifact.com/factchecks/list/?page="+str(i)
    page = requests.get(url,headers=head)
    print(page,end="\r")
    soup = BeautifulSoup(page.text, "html.parser")
    titles = soup.find_all(class_ = "m-statement--is-medium")
    for title in titles:
        newsName = title.find(class_="m-statement__name").text[1:-1]
        newsDesc = title.find(class_="m-statement__desc").text[1:-1]
        newsQuote = title.find(class_="m-statement__quote").text[2:-2]
        newsRate = title.find(class_="m-statement__meter").div.picture.img["alt"]
        newsRate = "mostly-false" if newsRate=="barely-true" else newsRate
        news = [newsName+" "+newsDesc+" "+newsQuote,newsRate]
        newsData.append(news)
        
df_politifact = pd.DataFrame(newsData,columns=['News','Rating'])
df_politifact.to_csv('data/politifact.csv',index=False)

In [5]:
newsData = list()
for i in range(221,1366):
    url = "https://www.snopes.com/fact-check/page/{}/".format(i)
    page = requests.get(url,headers=head)
    print(page,end="\r")
    soup = BeautifulSoup(page.text, "html.parser")
    titles = soup.find_all(class_="stretched-link")
    for title in titles[:-5]:
        checkUrl = title["href"]
        checkPage = requests.get(checkUrl,headers=head)
        checkSoup = BeautifulSoup(checkPage.text, "html.parser")
        if (checkSoup.find(class_="claim-text card-body") is None):
            continue
        newsQuote = checkSoup.find(class_="claim-text card-body").text.replace('\n','').replace('\t','')[:-1]
        if (checkSoup.find(string="Rating") is None):
            continue
        newsRate = checkSoup.find(string="Rating").parent.parent.parent.a['href']
        newsRate = newsRate.replace('https://www.snopes.com/fact-check/rating/','').replace("/","")
        news = [newsQuote,newsRate]
        newsData.append(news)

df_snopes = pd.DataFrame(newsData,columns=['News','Rating'])
df_snopes.to_csv('data/snopes.csv',index=False)

In [6]:
newsData = list()
for i in range(83,230):
    url = "https://www.truthorfiction.com/category/fact-checks/page/{}/".format(i)
    page = requests.get(url,headers=head)
    print(page,end="\r")
    if(str(page)[-5:-4]=='5'):
        i = i-1
        continue
    soup = BeautifulSoup(page.text, "html.parser")
    titles = soup.find_all("h2")[:-1]
    for title in titles:
        checkUrl = title.a["href"]
        checkPage = requests.get(checkUrl,headers=head)
        checkSoup = BeautifulSoup(checkPage.text, "html.parser")
        if (checkSoup.find(class_="claim-description") is None):
            continue
        newsQuote = checkSoup.find(class_="claim-description").text
        if (checkSoup.find(class_="rating-description") is None):
            continue
        newsRate = checkSoup.find(class_="rating-description").span.text
        news = [newsQuote,newsRate]
        newsData.append(news)

df_truthorfiction = pd.DataFrame(newsData,columns=['News','Rating'])
df_truthorfiction.to_csv('data/truthorfiction.csv',index=False)

In [12]:
newsData = list()
for i in range(76,121):
    url = "https://checkyourfact.com/page/{}/".format(i)
    page = requests.get(url,headers=head)
    print(page,end="\r")
    soup = BeautifulSoup(page.text, "html.parser")
    titles = soup.find("articles").find_all("a")
    for title in titles:
        checkUrl = "https://checkyourfact.com"+title['href']
        checkPage = requests.get(checkUrl,headers=head)
        checkSoup = BeautifulSoup(checkPage.text, "html.parser")
        if (checkSoup.p.text is None):
            continue
        newsQuote = checkSoup.p.text
        newsRate = checkSoup.find("strong")
        if (newsRate is None):
            newsRate = checkSoup.find("b")
        if (newsRate is None):
            continue
        newsRate = newsRate.text[9:]
        news = [newsQuote,newsRate]
        newsData.append(news)

df_checkyourfact = pd.DataFrame(newsData,columns=['News','Rating'])
df_checkyourfact.to_csv('data/checkyourfact.csv',index=False)